In [1]:
import pandas as pd
import os
file_name = "Stock_AnalysisQly_Details1.csv"
CSV_FILE = os.path.join(os.path.dirname(os.getcwd()), file_name)
print(CSV_FILE)
QSheet = pd.DataFrame.from_csv(CSV_FILE)

C:\Users\nandpara\PycharmProjects\Machine_Learning1\Stock_AnalysisQly_Details1.csv


c:\users\nandpara\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


# Declarations

In [2]:
Stock_Feature = ['RQ_1_2_PAT_PChg','RQ_1_2_Operating_Profit_PChg','RQ_1_2_Total_Exp_PChg','RQ_1_2_Ebit_PChg','RQ_1_2_Ebitda_PChg']
Stock_value = ['RQ_1_2_Close_PChg']
OP = 'RQ_1_2_Operating_Profit_PChg'
PAT = 'RQ_1_2_PAT_PChg'
Close = 'RQ_1_2_Close_PChg'

# Preprocessing

In [8]:
from sklearn.preprocessing import Imputer
import numpy as np
imputer = Imputer(strategy="median")
imputer.fit(QSheet)
imputer.statistics_
X = imputer.transform(QSheet)
NQSheet = pd.DataFrame(X, index=QSheet.index,columns=QSheet.columns)
#imputer now fills only the NaN values , inf value is still untreated....
NQSheet = NQSheet.replace(to_replace=[np.inf, -np.inf], value=0)

# Decision Tree - Generally no Scaling is required

# Removing Ouliers , 
 ## 1. Remove 5 % from Top and 5% from Bottom

In [9]:
FivePercent = int(len(QSheet)*0.2)
OP_Top_Outliers = NQSheet[OP].sort_values()[:FivePercent].index
OP_Bottom_Outliers = NQSheet[OP].sort_values()[-FivePercent:].index
PAT_Top_Outliers = NQSheet[PAT].sort_values()[:FivePercent].index
PAT_Bottom_Outliers = NQSheet[PAT].sort_values()[-FivePercent:].index
Outliers = set(list(OP_Top_Outliers) + list(OP_Bottom_Outliers) + list(PAT_Top_Outliers) + list(PAT_Bottom_Outliers))
NQSheet_No_Outliers = NQSheet.drop(Outliers)
len(NQSheet_No_Outliers)

726

In [13]:
train_Feature = NQSheet_No_Outliers[Stock_Feature][:5]
train_Value = NQSheet_No_Outliers[Stock_value][:5]

print(train_Feature)
print(train_Value)

from sklearn.tree import DecisionTreeRegressor
TreeReg = DecisionTreeRegressor()
TreeReg.fit(train_Feature, train_Value)
TreeReg.score(train_Feature, train_Value)

            RQ_1_2_PAT_PChg  RQ_1_2_Operating_Profit_PChg  \
KDDL              -1.790281                      7.635830   
ERIS              21.913043                     16.488846   
SORILINFRA       -62.323561                    -57.412399   
SOLARINDS         17.022809                     10.649351   
GEECEE           -55.863708                    -35.070306   

            RQ_1_2_Total_Exp_PChg  RQ_1_2_Ebit_PChg  RQ_1_2_Ebitda_PChg  
KDDL                    13.802906         12.357724           10.401003  
ERIS                    16.823886         13.622590           10.215254  
SORILINFRA             -39.166667        -59.382819          -43.411840  
SOLARINDS               -3.768867         13.764706           12.638759  
GEECEE                 -83.399734        -34.318555          -33.386581  
            RQ_1_2_Close_PChg
KDDL                24.063065
ERIS               -14.341982
SORILINFRA        -100.000000
SOLARINDS            5.572973
GEECEE               9.589598


1.0

In [12]:
from sklearn.tree import export_graphviz
export_graphviz(TreeReg, out_file='sample_decision_tree.dot', feature_names=Stock_Feature, class_names='Close_Price', rounded=True,filled=True)